In [1]:
import torch 
import numpy as np 
from torch.utils.data import DataLoader 
import matplotlib.pyplot as plt
import h5py 
import sys
models_path = "..\\Models architecture"
sys.path.append(models_path)
torch.manual_seed(0)

In [2]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

# Importing galaxies

In [3]:
path_cosmic_survey = 'C:\\Users\\noedi\\Desktop\\Stage été 2022\\Code stage\\Vae Galaxies\\Data\\cosmos_25.2_all_with_zphot.h5'
path_deepfield = 'C:\\Users\\noedi\\Desktop\\Stage été 2022\\Code stage\\Vae Galaxies\\Data\\cosmos_23.5_128_augmented_denoised.h5'

In [4]:
class Hdf_dataset(torch.utils.data.Dataset):
    def __init__(self, file):
        self.file = file
        self.hdf = h5py.File(file, 'r')
        self.datasets = list(self.hdf.keys())

    def __len__(self):
        size = len(self.hdf[self.datasets[0]])
        return size

    def __getitem__(self, idx):
    
        if len(self.datasets) == 2:
            x = self.hdf[self.datasets[0]][idx]
            z = self.hdf[self.datasets[1]][idx]
            return x, z
        
        else:
            x = self.hdf[self.datasets[0]][idx]
            return x

In [5]:
cosmic_survey_dataset = Hdf_dataset(path_cosmic_survey)
deepfield_dataset = Hdf_dataset(path_deepfield)

In [6]:
# Change cosmic_survey_dataset by deepfield_dataset if you want to train with Model_128vae 
data_size = len(cosmic_survey_dataset) # <-- change dataset if needed ! 
train_size = int(0.9 * data_size)
val_size = data_size - train_size

Generator = torch.Generator()
Generator.manual_seed(0)
train_set, val_set = torch.utils.data.random_split(cosmic_survey_dataset, [train_size, val_size], generator = Generator) # <-- change dataset if needed ! 

In [7]:
batchsize = 64
train_loader = DataLoader(train_set, batch_size = batchsize)
val_loader = DataLoader(val_set, batch_size = batchsize)

# Training Time
To train, `vae.train_time(train_loader, val_loader, epochs = 100, learning_rate = 1e-3 , beta = 0.1)`.  
$\beta$ can be a scalar or a table of values (list, array, tensor). Just make sure `len(beta) = epochs`.  
If you are working with the _fancy_cvae_ architecture `vae.train_time(*args, k=1000)` has an extra hyperparameter $k$ which is just a multiplier of the loss function between the true redshift value and its prediction (which is part of the general loss). Hence, with a greater value for $k$ neural network will learn better predictions of the redshifts. 

In [8]:
# Importing the model: Model_128vae, Model_158vae, cvae, cvae2, fancy_cvae
from cvae2 import VariationalAutoencoder

In [9]:
# Network parameters (for cvae2, nc= number of channels = 1, for cvae and fancy_cvae, nc = number of channels= 2)
z_dim =  32
vae = VariationalAutoencoder(z_dim).to(device)

# Training parameters
num_epochs = 100
lr = 1e-4
Beta = 1 # Beta-vae 
vae.train()
train_loss, val_loss, mse, kl = vae.train_time(train_loader, val_loader, epochs = num_epochs, learning_rate = lr, beta = Beta)

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Saving the weights and the loss 

In [ ]:
# Automatically creates the name of the file 
hyperparameters = {'batch size': batchsize, 'epochs': num_epochs, 'beta': Beta, 'learning rate': lr, 'z_dim': z_dim}

if type(Beta) == torch.Tensor or type(Beta) == np.ndarray or type(Beta) == list:
    beta_behaviour = input("Enter a word to describe beta's behaviour").replace(" ","")
    loss_file_name = "z"+str(z_dim)+"_beta"+beta_behaviour+"_loss.pt"
    weights_file_name = "z"+str(z_dim)+"_beta"+beta_behaviour+"_weights.pt"
else: 
    loss_file_name = "z"+str(z_dim)+"_beta"+str(Beta)+"_loss.pt"
    weights_file_name = "z"+str(z_dim)+"_beta"+str(Beta)+"_weights.pt"

data_training_cosmic_survey = "..\\Data\\" 
data_training_deepfield = "..\\Data\\"

weights_path = data_training_cosmic_survey + weights_file_name
loss_path = data_training_cosmic_survey + loss_file_name

# torch.save([vae.state_dict(), hyperparameters], weights_path)
# torch.save([train_loss, val_loss, mse, kl, hyperparameters], loss_path)